# LCLS cu_bc2 csr_wake creation

In [ ]:
from pytao import evaluate_tao
from distgen import Generator
from pmd_beamphysics import ParticleGroup, particle_paths

from h5py import File
import numpy as np
import os
import tempfile 

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (12,8)
%config InlineBackend.figure_format = 'retina'

c_light = 299792458.

In [ ]:
# Test archive file

AFILE0 = 'bmad_beam_e29b428eeafe83372a5dbf0f437a0de0.h5'
#AFILE0 = 'bmad_beam_3ef270fdbea59d7237bf619276db0bbe.h5'

In [ ]:
def get_bunch(afile, ix=3):
    with File(afile, 'r') as h5:
        ppaths = particle_paths(h5)
        P = ParticleGroup(h5[ppaths[ix]])
    return P
Pi= get_bunch(AFILE0, ix=3)
Pf= get_bunch(AFILE0, ix=4)

In [ ]:
Pf.plot('t', 'p')

In [ ]:
P0 = Pi.copy()
P0.t -= P0['mean_t']
P0.plot('t', 'p')

In [ ]:
P0.write_bmad('BEGBC2.beam0', t_ref=P0['mean_t'], p0c=5e9)

In [ ]:
P1 = ParticleGroup('BX23.h5')
P1.plot('delta_t', 'p')
P1['norm_emit_x']

In [ ]:
P1.plot('x', 'px')

In [ ]:
with tempfile.TemporaryDirectory() as fp:
    fp.write(b'a')

In [ ]:
ROOT = os.path.abspath(os.getcwd())
APATH=os.path.join(ROOT, 'archive')
INIT = os.path.join(ROOT, 'template/tao.init')
os.path.exists(INIT)

In [ ]:
%%time

def calc1(afile):
    
    tdir = tempfile.TemporaryDirectory()
    pfile= os.path.join(tdir.name, 'BC2BEG.beam0')
      
    P0 = get_bunch(afile, ix=3) # BC@BEG
    P0.write_bmad(pfile, t_ref=P0['mean_t'], p0c=5e9)
    
    # Tao
    res = evaluate_tao(settings={
                    'bmad_com:csr_and_space_charge_on': True,
                    'csr_param:write_csr_wake':True,
                    'csr_param:ds_track_step': 0.001,
                    'csr_param:n_bin': 200,
                    'beam:beam_saved_at': 'BEG_BX24,BX24',
                    'beam_init:position_file': pfile},
             run_commands=[
                 'set ele * space_charge_method = off',
                 'set ele BX24:DM23B CSR_METHOD  = 1_dim',
                 'set global track_type=beam'],
             expressions=['beam::norm_emit.x[ENDBC2]', 'beam::norm_emit.y[ENDBC2]', 'beam::sigma.z[ENDBC2]'],
             beam_archive_path='archive',
             archive_csr_wake=True,                       
             input_file=INIT, ploton=False)
    
    res['original_archive'] = afile
    
    return res
    
RES = calc1(AFILE0)  
RES

In [ ]:
!ls -ahl archive

In [ ]:
from pytao.misc.csr import read_csr_wake_data_h5

In [ ]:
with File(RES['beam_archive'], 'r') as h5:
    print(list(h5))
    print(dict(h5['data']['00002'].attrs))
    cdat = read_csr_wake_data_h5(h5['csr_wake'])

In [ ]:
cdat.keys()

In [ ]:
dat = np.concatenate([cdat[key]['data'] for key in list(cdat) ])

In [ ]:
sdat = np.concatenate([cdat[key]['s_positions'] for key in list(cdat) ])
len(sdat)

In [ ]:
def plot1(step=0):
    
    fig, ax = plt.subplots()
    z = dat[step,:,0]*1e3
    ax.set_xlabel('z (mm)')
    ax.set_ylabel('CSR Kick/m')
    density =  dat[step,:,1]
    kick = dat[step,:,2]
    
    zmin = np.min(dat[:,:,0])*1e3
    zmax = np.max(dat[:,:,0])*1e3
    
    
    avkick = np.sum(kick*density)/np.sum(density)
    stdkick = np.sqrt(np.sum( kick**2*density)/np.sum(density) - avkick**2)
    plt.plot([zmin, zmax], 2*[avkick], linestyle='dashed', color='black')
    plt.plot([zmin, zmax], 2*[stdkick+avkick],   linestyle='dotted', color='grey')
    plt.plot([zmin, zmax], 2*[-stdkick+avkick ], linestyle='dotted', color='grey')
    
    ax.plot(z, kick, color='black')
    
    kmin = np.min(dat[:,:,2])
    kmax = np.max(dat[:,:,2])
    
    ax.set_ylim(kmin, kmax)
    ax2 = ax.twinx()
    ax2.set_ylabel('density')
    ax2.plot(z, density, color='red')
    
plot1(step=-1)

In [ ]:
from ipywidgets import interact

interact(plot1, step=(0, len(dat)-1, 1) )

In [ ]:
def stats(step=0):
    
 
    z = dat[step,:,0]
    zmax = z.max()
    zmin = z.min()
    nz = len(z)
    dz = z.ptp()/(nz-1)
    density =  dat[step,:,1]
    qtot = np.sum(density)*dz
    # Normalize
    density /= np.sum(density)    
    
    avz = np.sum(z*density)
    stdz = np.sqrt(np.sum(z**2*density) - avz**2)
    
    kick = dat[step,:,2]    
    avkick = np.sum(kick*density)
    stdkick = np.sqrt(np.sum( kick**2*density) - avkick**2)
    return avkick, stdkick, avz, stdz

In [ ]:
STATS = np.array([stats(i) for i in range(len(dat))])

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel('s (m)')
ax.set_ylabel('CSR Kick/m')
ax.plot(sdat, STATS[:,0], color='black', label='Average Wake')
ax.plot(sdat, STATS[:,1], color='red', label='std Wake')

ax2 = ax.twinx()
ax2.plot(sdat, STATS[:,3]*1e15/c_light, color='blue', label='$\sigma_z/c (fs)$')
ax2.set_ylabel('$\sigma_z/c (fs)$')
ax.legend()